### Introduction to contravariant logging

We decided to use contravariant logging approach. I use here `co-log` lib naming convention because that is what we have in the code base now (`cardano-node` uses `contra-tracer`).

Some initial boilerplate imports and language extensions:

In [55]:
:ext GeneralizedNewtypeDeriving
:ext BlockArguments

import Control.Concurrent (forkIO, threadDelay)
import Data.Foldable (for_)
import Control.Monad (when)

Let's define the core logging data type which is just a logging function `msg -> m ()` (for example `Text -> IO ()`). We wrap in a `newtype` and call it `LogAction`:

In [2]:
newtype LogAction m msg = LogAction (msg -> m ())

-- Let's define a handy runner which just applies the function.
log :: LogAction m msg -> msg -> m ()
log (LogAction l) = l

For example `putStrLn` can be easily turned into a log action and it is `String` logging action in `IO`:

In [3]:
consoleLogAction :: LogAction IO String
consoleLogAction = LogAction putStrLn

We can also easily implement a file logger (but please don't use it in production ;-):

In [4]:
fileLogAction :: FilePath -> LogAction IO String
fileLogAction filePath = LogAction \msg -> appendFile filePath (msg <> "\n")

#### Combining actions "veritically"

> Note: I used "vertically" in the title because we can imagine that we have a logging pipeline - `log entry -> log transformation / adjustment -> log consumers`. From the application point of view we have a single entry point to this pipeline.

If we have loggers which work with the same message type (`String` in our case) I think that we have pretty natural way of "combining" the logging actions - we are nearly able to use `deriving newtype` for that and use `->` and `m ()` underlying instances but let's write this composition by hand... I mean please implement `<>` and `mempty`: 

In [5]:
instance (Applicative m) => Semigroup (LogAction m msg) where
  LogAction log1 <> LogAction log2 = LogAction \msg -> ???
  
instance (Applicative m) => Monoid (LogAction m msg) where
  mempty = LogAction \msg -> ??

We should be able to compose multiple loggers now:

In [6]:
appLogAction = consoleLogAction <> fileLogAction "app.log"

So let's log something..

In [7]:
log appLogAction "Hello World!"

Hello World!

and check if both loggers have consumed the message: 

In [8]:
readFile "app.log" >>= putStrLn

Hello World!

#### "Horizontal" adjustments

##### Message type adaptation

Let's imagine that we want to attach pretty standard attribute to the log messages which are useful during debugging which is `Severity`. `co-log` provides this data type for us:

In [9]:
data Severity = Debug | Info | Warn | Error
  deriving (Eq, Ord, Show)

Now we can introduce a bit richer message structure plus some formatting helper:

In [10]:
data Message = Message { msgSeverity :: Severity, msgText :: String }

fmtMessage :: Message -> String
fmtMessage (Message severity msg) = "[" <> show severity <> "] " <> msg 

Can we use our previously defined `consoleLogAction` or `fileLogAction` but using this richer `Message` structure? Please try to write this adapter given `fmtMessage` function:

In [11]:
adaptStrLogAction :: LogAction m String -> LogAction m Message
adaptStrLogAction (LogAction logAction) = LogAction ??

appLogAction' :: LogAction IO Message
appLogAction' = adaptStrLogAction appLogAction

log appLogAction' (Message Debug "Adapted string logger test")

[Debug] Adapted string logger test

If we generalize the above adaptation API it forms a bit non intuitive interface which is called contravariant functor:

In [12]:
class Contravariant f where
  contramap :: (b -> a) -> f a -> f b

We can read the `contramap` operation semantics in our context like:

> Given a `LogAction m a` (let swe have logger which consumes a `String` at hand) and a function which turns `b` into `a` (we have `fmtMessage` which turns `Message` into `String`) we should be able to build a `LogAction m b` (in our case new `LogAction` gonna consume `Message` values).

Could you please implement `Contravariant` for `LogAction`:

In [13]:
instance Contravariant (LogAction m) where
  contramap f (LogAction logAction) = LogAction ??
  
log (contramap fmtMessage consoleLogAction) (Message Debug "Adapted using contramap")

[Debug] Adapted using contramap

##### Logs context anotation

What is really interesting is that we can use the above API also to annotate the logs with context. Let's imagine that we have a subsystem entry point like:

In [49]:
worker :: LogAction IO Message -> IO ()
worker logAction = do
  log logAction (Message Debug "Initializing")
  threadDelay (2 * 10^6)
  log logAction (Message Info "Processing")
  threadDelay (2 * 10^6)
  log logAction (Message Debug "Shutting down")

Please note that the above `worker` logs messages related to it's own context because it doesn't know the full application level context which it is "embedded" in - it doesn't know its parents, `id` etc.

Now let's create an application which spawns a bunch of this workers:

In [50]:
app logAction = do
  log logAction (Message Debug "Initializing application.")
  log logAction (Message Info "Starting workers swarm")
  for_ [1,2,3,4] \i -> do
    let
      workerLogAction = contramap (\(Message sev msg) -> Message sev $ "[Worker-" <> show i <> "] " <> msg) logAction
    threadDelay (10^5)
    forkIO $ worker workerLogAction
  threadDelay (5 * 10^6)
  log appLogAction' (Message Info "Collecting results")
  log appLogAction' (Message Debug "Shutting down")

The above code uses really buggy and naive approach to concurrent logging but should work for this simple example. Let's run it and check what kind of logs we gonna get.

In [51]:
app appLogAction'

[Debug] Initializing application.
[Info] Starting workers swarm
[Debug] [Worker-1] Initializing
[Debug] [Worker-2] Initializing
[Debug] [Worker-3] Initializing
[Debug] [Worker-4] Initializing
[Info] [Worker-1] Processing
[Info] [Worker-2] Processing
[Info] [Worker-3] Processing
[Info] [Worker-4] Processing
[Debug] [Worker-1] Shutting down
[Debug] [Worker-2] Shutting down
[Debug] [Worker-3] Shutting down
[Debug] [Worker-4] Shutting down
[Info] Collecting results
[Debug] Shutting down

So we were able to annotate the internal subsystem logs with the context from the outside. Nice!

We can also use the same based `contramap` based technique to also annotate "semantic" logs - you can imagine that we can have:

In [42]:
data WorkerLog = Initializing | Processing | ShuttingDown
  deriving Show

type WorkerId = Int

data AppLog = WorkerLog WorkerId WorkerLog | InitializaingApp | ShuttingDownApp
  deriving Show
  
-- Defined just for demonstration purposes
appLogAction'' :: LogAction IO AppLog
appLogAction'' = LogAction (const $ pure ())

Now we can turn our application level logger `LogAction m AppLog` into `LogAction m WorkerLog` just by using `contramap`:

In [44]:
workerLogAction :: WorkerId -> LogAction IO WorkerLog
workerLogAction i = contramap (WorkerLog i) appLogAction''

And this final logger could be passed to the worker subsystem.

##### Message filtering

We can ask ourselves what is the value of the `Message` type if we just turn it into the `String` - we could just use plain `String` and skip it all together. That is not necessarly true - `Severity` value which we carry together with the message can be used for filtering messages so we can have use verbosity level.
Let's implement such a filtering function - given a severity value please log only messages which have at least such a level:

In [57]:
setVerbosity :: Applicative m => Severity -> LogAction m Message -> LogAction m Message
setVerbosity sev (LogAction f) = LogAction $ \msg@(Message sev' _) -> when (sev <= sev') $ f msg

Now we can test it using our previously defined `app`:

In [59]:
app (setVerbosity Info appLogAction')

[Info] Starting workers swarm
[Info] [Worker-1] Processing
[Info] [Worker-2] Processing
[Info] [Worker-3] Processing
[Info] [Worker-4] Processing
[Info] Collecting results
[Debug] Shutting down